# nn.Module————模型构造

### 1.最常规模型
forward(self,X)相当于重载( )，此即需要继承自nn.Module的原因之一<p>
init需要自己定义各层函数
重载foraward(self,X)&添加self层

In [1]:
import torch 
from torch import nn
input_size=728
hidden_size=256
output_size=10

class MLP(nn.Module):
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        self.to_hidden=nn.Linear(input_size,hidden_size)
        self.activate=nn.ReLU()
        self.to_output=nn.Linear(hidden_size,output_size)
    def forward(self,X):                                       
        return self.to_output(self.activate(self.to_hidden(X)))

In [2]:
net=MLP()
X=torch.rand(728)
net(X)

tensor([ 0.3337,  0.1345,  0.0134,  0.1981, -0.0540,  0.1329, -0.0345, -0.0508,
         0.0121,  0.0565], grad_fn=<AddBackward0>)

### 2.封装好的nn.Module子类：Sequential

<mark>源码

In [3]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): # 如果传入的是一个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)  # add_module方法会将module添加进self._modules(一个OrderedDict)
        else:  # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    def forward(self, input):
        # self._modules返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成员
        for module in self._modules.values():
            input = module(input)
        return input


enumerate(< list>):将list元素配合索引组成元组，而后构造新列表

In [4]:
net=nn.Sequential(
    nn.Linear(input_size,hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size,output_size)
)
net(X)

tensor([-0.0464, -0.0626,  0.1564,  0.0953, -0.1219, -0.0788, -0.2519,  0.0688,
         0.2111, -0.0161], grad_fn=<AddBackward0>)

### 3.演练

In [5]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)#--------------------------------------------s输入层到隐藏层
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)#---------------------激活函数用仿射和ReLU复合

        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)#-------------------------------隐藏层到输出层
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:#--------------------------输出层最终处理
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()


TENSOR.norm()：求TENSOR的模长(平方根号)；item():将tensor转换成python数

In [6]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-1.1271, grad_fn=<SumBackward0>)

In [7]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU()) 

    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())
#net有三个隐藏层
X = torch.rand(2, 40)
print(net)
net(X)


Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-5.9302, grad_fn=<SumBackward0>)

<mark>nn.Sequential的主要作用是实现各种模型的“拼接”<p>
<mark>复杂模型还是要继承nn.Module之后自己写

### 4.通过ParameterList/Dict实现外层选择内部模型



我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。<p>

在4.2节（模型参数的访问、初始化和共享）中介绍了Parameter类其实是Tensor的子类，如果一个Tensor是Parameter，那么它会自动被添加到模型的参数列表里。所以在自定义含模型参数的层时，我们应该将参数定义成Parameter，除了像4.2.1节那样直接定义成Parameter类外，还可以使用ParameterList和ParameterDict分别定义参数的列表和字典。<p>
<mark>之所以不能用list/dict是因为只有ParameterList/Dict才能进入net.parameters()<mark\><p>
ParameterList接收一个Parameter实例的列表作为输入然后得到一个参数列表，使用的时候可以用索引来访问某个参数，另外也可以使用append和extend在列表后面新增参数。

而ParameterDict接收一个Parameter实例的字典作为输入然后得到一个参数字典，然后可以按照字典的规则使用了。例如使用update()新增参数，使用keys()返回所有键值，使用items()返回所有键值对等等，可参考官方文档。

In [8]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'linear1': nn.Parameter(torch.randn(4, 4)),
                'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))}) # 新增

    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

net = MyDictDense()
print(net)


MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


<mark> ParameterDict的目的：这样就可以根据传入的键值来进行不同的前向传播：

In [9]:
x = torch.ones(1, 4)
print(net(x, 'linear1'))
print(net(x, 'linear2'))
print(net(x, 'linear3'))


tensor([[-1.1469,  2.0551, -2.9653, -0.9012]], grad_fn=<MmBackward0>)
tensor([[-2.1885]], grad_fn=<MmBackward0>)
tensor([[-0.0366, -0.3632]], grad_fn=<MmBackward0>)
